In [1]:
import pandas as pd
import numpy as np
from darts import TimeSeries
from darts.models import RegressionModel
from feature_engine.creation import CyclicalFeatures
from feature_engine.datetime import DatetimeFeatures
from feature_engine.imputation import DropMissingData
from feature_engine.selection import DropFeatures
from feature_engine.timeseries.forecasting import LagFeatures
from feature_engine.timeseries.forecasting import LagFeatures, WindowFeatures
from darts.metrics import mape, smape, mae
from darts.models import RNNModel
from darts.models import BlockRNNModel
from pytorch_lightning.callbacks import EarlyStopping
from torchmetrics import MeanAbsolutePercentageError
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import holidays

The `LightGBM` module could not be imported. To enable LightGBM support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md
The `Prophet` module could not be imported. To enable Prophet support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md
The `CatBoost` module could not be imported. To enable CatBoost support in Darts, follow the detailed instructions in the installation guide: https://github.com/unit8co/darts/blob/master/INSTALL.md


In [2]:
df = pd.read_csv(r"D:\Git\darts-pipeline\data\raw\rossmann_train.csv")

C:\Users\steng\AppData\Local\Temp\ipykernel_25112\1630001524.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"D:\Git\darts-pipeline\data\raw\rossmann_train.csv")


In [3]:
df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [4]:
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

mask = df["Date"].isna()
df.loc[mask, "Date"] = pd.to_datetime(
    df.loc[mask, "Date"], format="%d-%m-%Y", errors="coerce"
)

In [5]:
df.dropna(inplace=True)
df

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [6]:
df.set_index("Date", inplace=True)

In [7]:
dtf = DatetimeFeatures(
    variables="index",
    features_to_extract=[
        "month",
        "week",
        "day_of_week",
        "day_of_month",
        "day_of_year",
        "hour",
        "weekend",
    ],
)

df = dtf.fit_transform(df)
df.head()

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,month,week,day_of_week,day_of_month,day_of_year,hour,weekend
Date,,,,,,,,,,,,,,,
2015-07-31,1,5,5263,555,1,1,0,1,7,31,4,31,212,0,0
2015-07-31,2,5,6064,625,1,1,0,1,7,31,4,31,212,0,0
2015-07-31,3,5,8314,821,1,1,0,1,7,31,4,31,212,0,0
2015-07-31,4,5,13995,1498,1,1,0,1,7,31,4,31,212,0,0
2015-07-31,5,5,4822,559,1,1,0,1,7,31,4,31,212,0,0


In [8]:
dfs = []

df = df.reset_index()

grouped = df.groupby("Store")

for name, group in grouped:
    group_copy = group.copy()

    group_copy = group_copy.set_index("Date")

    cyclicf = CyclicalFeatures(
        variables=["month", "week", "day_of_week", "day_of_month", "day_of_year"],
        drop_original=False,
    )

    group = cyclicf.fit_transform(group)

    dfs.append(group_copy)

df_transformed = pd.concat(dfs)
df = df_transformed.reset_index()

df = df.set_index("Date")


df.shape

(1017209, 15)

In [9]:
df

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,month,week,day_of_week,day_of_month,day_of_year,hour,weekend
Date,,,,,,,,,,,,,,,
2015-07-31,1,5,5263,555,1,1,0,1,7,31,4,31,212,0,0
2015-07-30,1,4,5020,546,1,1,0,1,7,31,3,30,211,0,0
2015-07-29,1,3,4782,523,1,1,0,1,7,31,2,29,210,0,0
2015-07-28,1,2,5011,560,1,1,0,1,7,31,1,28,209,0,0
2015-07-27,1,1,6102,612,1,1,0,1,7,31,0,27,208,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-01-05,1115,6,4771,339,1,0,0,1,1,1,5,5,5,0,1
2013-01-04,1115,5,4540,326,1,0,0,1,1,1,4,4,4,0,0
2013-01-03,1115,4,4297,300,1,0,0,1,1,1,3,3,3,0,0


In [10]:
dfs = []

df = df.reset_index()

grouped = df.groupby("Store")

for name, group in grouped:
    group_copy = group.copy()

    group_copy = group_copy.set_index("Date")

    lagf = LagFeatures(
        variables=["Sales", "Customers", "Open", "Promo"],
        freq=[str(x) + "D" for x in range(1, 20)],
        missing_values="ignore",
    )

    group_copy = lagf.fit_transform(group_copy)
    dfs.append(group_copy)

df_transformed = pd.concat(dfs)
df = df_transformed.reset_index()

df = df.set_index("Date")

df.shape

(1017209, 91)

In [11]:
dfs = []

df = df.reset_index()

grouped = df.groupby("Store")

for name, group in grouped:
    group_copy = group.copy()

    group_copy = group_copy.set_index("Date")

    winf = WindowFeatures(
        variables=["Sales", "Customers", "Open", "Promo"],
        window=["7D", "14D"],
        freq="D",
        missing_values="ignore",
    )

    group_copy = winf.fit_transform(group_copy)
    dfs.append(group_copy)

df_transformed = pd.concat(dfs)
df = df_transformed.reset_index()

df = df.set_index("Date")

df.shape

(1017209, 99)

In [12]:
del df_transformed
del group_copy
del grouped
del dfs

In [13]:
print(df.index.min(), df.index.max())

2013-01-01 00:00:00 2015-07-31 00:00:00


In [14]:
df = df.reset_index()
df = df.dropna()
df = df[df["Store"].isin(range(1, 10))]

In [15]:
columns = df.columns
print(list(columns))

['Date', 'Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'month', 'week', 'day_of_week', 'day_of_month', 'day_of_year', 'hour', 'weekend', 'Sales_lag_1D', 'Customers_lag_1D', 'Open_lag_1D', 'Promo_lag_1D', 'Sales_lag_2D', 'Customers_lag_2D', 'Open_lag_2D', 'Promo_lag_2D', 'Sales_lag_3D', 'Customers_lag_3D', 'Open_lag_3D', 'Promo_lag_3D', 'Sales_lag_4D', 'Customers_lag_4D', 'Open_lag_4D', 'Promo_lag_4D', 'Sales_lag_5D', 'Customers_lag_5D', 'Open_lag_5D', 'Promo_lag_5D', 'Sales_lag_6D', 'Customers_lag_6D', 'Open_lag_6D', 'Promo_lag_6D', 'Sales_lag_7D', 'Customers_lag_7D', 'Open_lag_7D', 'Promo_lag_7D', 'Sales_lag_8D', 'Customers_lag_8D', 'Open_lag_8D', 'Promo_lag_8D', 'Sales_lag_9D', 'Customers_lag_9D', 'Open_lag_9D', 'Promo_lag_9D', 'Sales_lag_10D', 'Customers_lag_10D', 'Open_lag_10D', 'Promo_lag_10D', 'Sales_lag_11D', 'Customers_lag_11D', 'Open_lag_11D', 'Promo_lag_11D', 'Sales_lag_12D', 'Customers_lag_12D', 'Open_lag_12D', 'Promo_lag_12D',

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8307 entries, 19 to 8477
Data columns (total 100 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Date                       8307 non-null   datetime64[ns]
 1   Store                      8307 non-null   int64         
 2   DayOfWeek                  8307 non-null   int64         
 3   Sales                      8307 non-null   int64         
 4   Customers                  8307 non-null   int64         
 5   Open                       8307 non-null   int64         
 6   Promo                      8307 non-null   int64         
 7   StateHoliday               8307 non-null   object        
 8   SchoolHoliday              8307 non-null   int64         
 9   month                      8307 non-null   int64         
 10  week                       8307 non-null   int64         
 11  day_of_week                8307 non-null   int64         
 12  day_

In [17]:
for col in df.columns:
    if col != "Date":
        df[col] = df[col].apply(
            lambda x: x if pd.to_numeric(x, errors="coerce") is not np.nan else 0
        )

df = df.drop(columns="StateHoliday")

In [18]:
df.head(20)

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday,month,week,...,Open_lag_19D,Promo_lag_19D,Sales_window_7D_mean,Customers_window_7D_mean,Open_window_7D_mean,Promo_window_7D_mean,Sales_window_14D_mean,Customers_window_14D_mean,Open_window_14D_mean,Promo_window_14D_mean
19,2013-01-20,1,7,0,0,0,0,0,1,3,...,0.0,0.0,3711.142857,488.142857,0.857143,0.000000,4209.285714,523.928571,0.857143,0.357143
20,2013-01-21,1,1,5394,607,1,1,0,1,4,...,1.0,0.0,3711.142857,488.142857,0.857143,0.000000,4209.285714,523.928571,0.857143,0.357143
21,2013-01-22,1,2,5720,667,1,1,0,1,4,...,1.0,0.0,3807.857143,486.857143,0.857143,0.142857,4082.000000,511.214286,0.857143,0.357143
22,2013-01-23,1,3,5578,627,1,1,0,1,4,...,1.0,0.0,4067.857143,509.000000,0.857143,0.285714,4092.000000,512.142857,0.857143,0.357143
23,2013-01-24,1,4,5195,645,1,1,0,1,4,...,1.0,0.0,4292.142857,522.857143,0.857143,0.428571,4099.642857,512.214286,0.857143,0.357143
24,2013-01-25,1,5,5586,676,1,1,0,1,4,...,0.0,0.0,4456.571429,543.142857,0.857143,0.571429,4121.285714,514.357143,0.857143,0.357143
25,2013-01-26,1,6,5598,640,1,0,0,1,4,...,1.0,1.0,4665.000000,558.571429,0.857143,0.714286,4171.642857,520.357143,0.857143,0.357143
26,2013-01-27,1,7,0,0,0,0,0,1,4,...,1.0,1.0,4724.428571,551.714286,0.857143,0.714286,4217.785714,519.928571,0.857143,0.357143
27,2013-01-28,1,1,4055,549,1,0,0,1,5,...,1.0,1.0,4724.428571,551.714286,0.857143,0.714286,4217.785714,519.928571,0.857143,0.357143
28,2013-01-29,1,2,3725,522,1,0,0,1,5,...,1.0,1.0,4533.142857,543.428571,0.857143,0.571429,4170.500000,515.142857,0.857143,0.357143


In [19]:
df.shape

(8307, 99)

In [20]:
df.to_parquet(r"D:\Git\darts-pipeline\data\dataset\rossmann.parquet")